# Experiment on AAI Model Architectures
- Created: 10 Mar 2020
- Data: param_draft2
Increase data size and change model architecture

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from os.path import join
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import callbacks
from time import time
from datetime import datetime
import shutil 
import argparse
from functools import partial
import pickle
from IPython.display import Audio

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Reshape, GRU, InputLayer, AlphaDropout, Activation, BatchNormalization, Dropout, Flatten, Dense, Bidirectional, LSTM, Conv1D, SpatialDropout1D, Concatenate, Multiply, Add

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder

In [2]:
np_load_old = partial(np.load)
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [3]:
tf.random.set_seed(42)

## Metric Function

In [4]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def R2(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred), axis = 0) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true, axis = 0)), axis = 0) 
    return K.mean(1 - (SS_res/SS_tot), axis=0)

## Utility Functions

In [5]:
#Function for preprocessing data
def delete_params(params):
    '''
    This function remove JX, WC, TRX, TRY, and MS1,2,3 paramter
    '''
    DEL_PARAMS_LIST = [2,8,15,16,21,22,23]
    return np.delete(params,DEL_PARAMS_LIST , axis=1)

In [6]:
def compute_rmse(actual,pred, axis=1):
    # Compute RMSE by row (axis=1) result in rmse of each data
    # Compute RMSE by column (axis=0) result in rmse of each label
    return np.sqrt((np.square(actual - pred)).mean(axis=axis))

In [7]:
def evaluate_model(y_pred, label):
    y_pred = enc.inverse_transform(y_pred)
    label = enc.inverse_transform(label)
#     print('Confusion Matrix')
#     print(confusion_matrix(label, y_pred))
    print('Classification Report')
    print(classification_report(label, y_pred))

In [8]:
def compute_se(actual,pred):
    return np.sqrt((np.square(actual - pred)))

## Load Data 

In [9]:
def prep_data():

    # load data from preprocess pipeline
    dataset = np.load(join('../../data/d_dataset_p2/prep_data_13','training_subsets.npz'))
    X_train =dataset['X_train']
    y_train= dataset['y_train']
    z_train= dataset['z_train']
    X_val = dataset['X_val']
    y_val = dataset['y_val']
    z_val = dataset['z_val']
    X_test = dataset['X_test']
    y_test = dataset['y_test']
    z_test = dataset['z_test']
    a_test = dataset['audio_test']
    a_val = dataset['audio_val']
    
    y_train = delete_params(y_train)
    y_val = delete_params(y_val)
    y_test = delete_params(y_test)

    print('Train features and labels %s %s'%(str(X_train.shape),str(y_train.shape)))
    print('Validating features and labels %s %s'%(str(X_val.shape),str(y_val.shape)))
    print('Test features and labels %s %s'%(str(X_test.shape),str(y_test.shape)))

    return X_train, X_val, X_test, y_train, y_val, y_test, z_train, z_val, z_test, a_test, a_val

In [10]:
X_train, X_val, X_test, y_train, y_val, y_test, z_train, z_val, z_test, a_test, a_val = prep_data()

Train features and labels (76260, 24, 39) (76260, 17)
Validating features and labels (9340, 24, 39) (9340, 17)
Test features and labels (9338, 24, 39) (9338, 17)


In [11]:
print(set(z_test))

{'a', 'u', 'o', 'y', '2', 'U', '9', 'i', 'O', 'E:', 'E', 'e', '@', 'A'}


In [12]:
print('Train features and labels {} {} {}'.format(str(X_train.shape),str(y_train.shape),str(z_train.shape)))
print('Validating features and labels {} {} {}'.format(str(X_val.shape),str(y_val.shape),str(z_val.shape)))
print('Test features and labels {} {} {}'.format(str(X_test.shape),str(y_test.shape),str(z_test.shape)))

Train features and labels (76260, 24, 39) (76260, 17) (76260,)
Validating features and labels (9340, 24, 39) (9340, 17) (9340,)
Test features and labels (9338, 24, 39) (9338, 17) (9338,)


### One-hot label 

In [13]:
z_train_oe=z_train.reshape(-1,1)
z_val_oe=z_val.reshape(-1,1)
z_test_oe=z_test.reshape(-1,1)

enc = OneHotEncoder(handle_unknown = 'ignore')
enc.fit(z_train_oe)

z_train_oe = enc.transform(z_train_oe).toarray()
z_test_oe = enc.transform(z_test_oe).toarray()
z_val_oe = enc.transform(z_val_oe).toarray()

## 1.) Train Baseline Model
- Model without embedding feature

### Defined Model without Embedded Feature

In [14]:
N_OUTPUTS = 17

pLSTM = partial(LSTM, kernel_initializer='he_uniform', return_sequences=True)
pConv1D = partial(Conv1D, padding = 'same', activation = 'linear', kernel_initializer = 'he_uniform')

def init_senet(feature_layer=1, cnn_unit=128, first_kernel=11, res_unit=64, res_kernel=3,
    bilstm = 1, bilstm_unit=256, 
    dropout_rate=0.3,
    reduction_ratio = 2,
    activation_fn='relu'):


    def cnn_block(input_x, cnn_unit, kernel_size):
        x = pConv1D(cnn_unit, kernel_size=kernel_size)(input_x)
        x = BatchNormalization()(x)
        x = Activation(activation_fn)(x)
        return x

    def se_block(input_x):
        x = layers.GlobalAveragePooling1D()(input_x)
        channel_shape = getattr(x, '_shape_val')[-1]
        x = Reshape((1, channel_shape))(x)
        x = Dense(channel_shape // reduction_ratio, activation=activation_fn, kernel_initializer='he_uniform')(x)
        x = Dense(channel_shape, activation='tanh', kernel_initializer='he_uniform')(x)
        return x

    def residual_block(input_x):
        x = cnn_block(input_x, res_unit,kernel_size=res_kernel)
        x = pConv1D(res_unit, kernel_size=3)(x)
        x = BatchNormalization()(x)
        return x

    def se_res_block(input_x):
        res_x = residual_block(input_x)
        se_x = se_block(res_x)
        x = layers.Multiply()([res_x, se_x])
        x = Activation(activation_fn)(x)
        input_x = cnn_block(input_x, cnn_unit=res_unit, kernel_size=1)
        outputs = layers.Add()([x, input_x])
        return outputs

    def senet_nn(input_shape_1,input_shape_2):

        input_x = keras.Input(shape=(input_shape_1,input_shape_2))
        x = cnn_block(input_x, 128, first_kernel)
        for i in range(feature_layer):
            x = se_res_block(x)
        x = SpatialDropout1D(rate=dropout_rate)(x)
        for i in range(feature_layer):
            x = se_res_block(x)
            x = SpatialDropout1D(rate=dropout_rate)(x)
        x = cnn_block(x, cnn_unit=cnn_unit, kernel_size=1)
        x = SpatialDropout1D(rate=dropout_rate)(x)
        for i in range(bilstm-1):
            x = Bidirectional(pLSTM(bilstm_unit))(x)
            x = SpatialDropout1D(rate=dropout_rate)(x)
        x = Bidirectional(pLSTM(bilstm_unit, return_sequences=False))(x)
        x = Dropout(rate=dropout_rate)(x)
        outputs = Dense(N_OUTPUTS, activation='linear', kernel_initializer='he_uniform')(x)
        model = keras.Model(inputs=input_x, outputs=outputs)
        return model

    return senet_nn

In [15]:
model = init_senet()(24, 39)

In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 24, 39)]     0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 24, 128)      55040       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 24, 128)      512         conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 24, 128)      0           batch_normalization[0][0]        
______________________________________________________________________________________________

### Compile Model 

In [17]:
model.compile(optimizer='adam',loss='mse',metrics=[rmse, R2])

### Train Model 

In [18]:
PATIENCE = 10
BATCH_SIZE = 128
EPOCHS = 100

early = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=PATIENCE, verbose=1, mode='min', baseline=None, restore_best_weights=False)
callback_list = [early]

In [19]:
history = model.fit(X_train,y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val,y_val),
    callbacks=callback_list)

Train on 76260 samples, validate on 9340 samples
Epoch 1/100
76260/76260 [==============================] - 28s 364us/sample - loss: 0.1078 - rmse: 0.3056 - R2: 0.5640 - val_loss: 0.0460 - val_rmse: 0.1973 - val_R2: 0.8115
Epoch 2/100
76260/76260 [==============================] - 17s 224us/sample - loss: 0.0602 - rmse: 0.2267 - R2: 0.7557 - val_loss: 0.0393 - val_rmse: 0.1786 - val_R2: 0.8391
Epoch 3/100
76260/76260 [==============================] - 17s 224us/sample - loss: 0.0523 - rmse: 0.2096 - R2: 0.7883 - val_loss: 0.0354 - val_rmse: 0.1718 - val_R2: 0.8559
Epoch 4/100
76260/76260 [==============================] - 17s 223us/sample - loss: 0.0482 - rmse: 0.2005 - R2: 0.8051 - val_loss: 0.0344 - val_rmse: 0.1690 - val_R2: 0.8611
Epoch 5/100
76260/76260 [==============================] - 17s 224us/sample - loss: 0.0450 - rmse: 0.1939 - R2: 0.8176 - val_loss: 0.0313 - val_rmse: 0.1617 - val_R2: 0.8732
Epoch 6/100
76260/76260 [==============================] - 17s 224us/sample - los

In [20]:
model.save('model_baseline.hdf5')

### Evaluation 

In [21]:
model.evaluate(X_test,y_test,verbose=0)

[0.023504588644473655, 0.13791166, 0.8972408]

In [22]:
y_pred = model.predict(X_test)

In [23]:
err = compute_rmse(y_test,y_pred, axis=0)

In [24]:
print(err)

[0.15548429 0.13400488 0.1198775  0.13760332 0.15043165 0.19396482
 0.21009323 0.09899641 0.11879603 0.14950289 0.15809402 0.13600232
 0.14012452 0.18484101 0.16447261 0.14721315 0.16574249]


## 2.) Trained Embedding Layer 

### Defined Embedding Model

In [25]:
PHONETIC_OUTPUT = len(set(z_train))
EMBEDDED_FEATURE = 39

pLSTM = partial(LSTM, kernel_initializer='he_uniform', return_sequences=True)

def embeded_model(input_shape_1,input_shape_2):
    inputs = keras.Input(shape=(input_shape_1,input_shape_2))
    x = pLSTM(EMBEDDED_FEATURE)(inputs)
    x = Flatten()(x)
    outputs = Dense(PHONETIC_OUTPUT, activation='softmax', kernel_initializer='he_uniform')(x)
    return keras.Model(inputs=inputs, outputs=outputs)

In [26]:
embedded = embeded_model(24, 39)
embedded.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 24, 39)]          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 24, 39)            12324     
_________________________________________________________________
flatten (Flatten)            (None, 936)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 14)                13118     
Total params: 25,442
Trainable params: 25,442
Non-trainable params: 0
_________________________________________________________________


### Compile Model 

In [27]:
embedded.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

### Train Model 

In [28]:
PATIENCE = 10
BATCH_SIZE = 128
EPOCHS = 100

early = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=PATIENCE, verbose=1, mode='min', baseline=None, restore_best_weights=False)
callback_list = [early]

In [29]:
history = embedded.fit(X_train,z_train_oe,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val,z_val_oe),
    callbacks=callback_list)

Train on 76260 samples, validate on 9340 samples
Epoch 1/100
76260/76260 [==============================] - 6s 73us/sample - loss: 2.0241 - accuracy: 0.7599 - val_loss: 1.8864 - val_accuracy: 0.8793
Epoch 2/100
76260/76260 [==============================] - 3s 40us/sample - loss: 1.9031 - accuracy: 0.8616 - val_loss: 1.8546 - val_accuracy: 0.9063
Epoch 3/100
76260/76260 [==============================] - 3s 42us/sample - loss: 1.8748 - accuracy: 0.8879 - val_loss: 1.8424 - val_accuracy: 0.9171
Epoch 4/100
76260/76260 [==============================] - 3s 40us/sample - loss: 1.8602 - accuracy: 0.9015 - val_loss: 1.8379 - val_accuracy: 0.9215
Epoch 5/100
76260/76260 [==============================] - 3s 40us/sample - loss: 1.8497 - accuracy: 0.9114 - val_loss: 1.8309 - val_accuracy: 0.9275
Epoch 6/100
76260/76260 [==============================] - 3s 40us/sample - loss: 1.8420 - accuracy: 0.9187 - val_loss: 1.8290 - val_accuracy: 0.9288
Epoch 7/100
76260/76260 [==========================

In [30]:
embedded.save('embedded.hdf5')

### Evaluated Model 

In [31]:
z_pred = embedded.predict(X_test)
evaluate_model(z_pred, z_test_oe)

Classification Report
              precision    recall  f1-score   support

           2       0.95      0.97      0.96       701
           9       0.98      0.96      0.97       688
           @       0.97      0.94      0.96       635
           A       0.93      0.97      0.95       645
           E       0.93      0.95      0.94       697
          E:       0.95      0.95      0.95       655
           O       0.93      0.95      0.94       729
           U       0.92      0.87      0.90       687
           a       0.96      0.94      0.95       689
           e       0.94      0.95      0.95       644
           i       0.98      0.94      0.96       684
           o       0.95      0.87      0.90       634
           u       0.86      0.95      0.91       659
           y       0.93      0.96      0.94       591

    accuracy                           0.94      9338
   macro avg       0.94      0.94      0.94      9338
weighted avg       0.94      0.94      0.94      9338



## 3.) Model with Embedding Feature 

### Embedded input model

In [32]:
embedded = tf.keras.models.load_model('embedded.hdf5')

embedded_layers = tf.keras.Sequential()
for layer in embedded.layers[:-2]:
    layer.trainable = False
    embedded_layers.add(layer)
embedded_layers.summary()

N_OUTPUTS = 17

pLSTM = partial(LSTM, kernel_initializer='he_uniform', return_sequences=True)
pConv1D = partial(Conv1D, padding = 'same', activation = 'linear', kernel_initializer = 'he_uniform')

def init_senet_w_emb(feature_layer=1, cnn_unit=128, first_kernel=11, res_unit=64, res_kernel=3,
    bilstm = 1, bilstm_unit=256, 
    dropout_rate=0.3,
    reduction_ratio = 2,
    activation_fn='relu'):

    def cnn_block(input_x, cnn_unit, kernel_size):
        x = pConv1D(cnn_unit, kernel_size=kernel_size)(input_x)
        x = BatchNormalization()(x)
        x = Activation(activation_fn)(x)
        return x

    def se_block(input_x):
        x = layers.GlobalAveragePooling1D()(input_x)
        channel_shape = getattr(x, '_shape_val')[-1]
        x = Reshape((1, channel_shape))(x)
        x = Dense(channel_shape // reduction_ratio, activation=activation_fn, kernel_initializer='he_uniform')(x)
        x = Dense(channel_shape, activation='tanh', kernel_initializer='he_uniform')(x)
        return x

    def residual_block(input_x):
        x = cnn_block(input_x, res_unit,kernel_size=res_kernel)
        x = pConv1D(res_unit, kernel_size=3)(x)
        x = BatchNormalization()(x)
        return x

    def se_res_block(input_x):
        res_x = residual_block(input_x)
        se_x = se_block(res_x)
        x = layers.Multiply()([res_x, se_x])
        x = Activation(activation_fn)(x)
        input_x = cnn_block(input_x, cnn_unit=res_unit, kernel_size=1)
        x = layers.Add()([x, input_x])
        return x

    def senet_nn(input_shape_1,input_shape_2):

        input_x = keras.Input(shape=(input_shape_1,input_shape_2))
        embedded = embedded_layers(input_x)
        x = layers.Concatenate()([input_x, embedded])
        x = cnn_block(input_x, 128, first_kernel)

        for i in range(feature_layer):
            x = se_res_block(x)
        x = SpatialDropout1D(rate=dropout_rate)(x)
        for i in range(feature_layer):
            x = se_res_block(x)
            x = SpatialDropout1D(rate=dropout_rate)(x)
        x = cnn_block(x, cnn_unit=cnn_unit, kernel_size=1)
        x = SpatialDropout1D(rate=dropout_rate)(x)
        for i in range(bilstm-1):
            x = Bidirectional(pLSTM(bilstm_unit))(x)
            x = SpatialDropout1D(rate=dropout_rate)(x)
        x = Bidirectional(pLSTM(bilstm_unit, return_sequences=False))(x)
        x = Dropout(rate=dropout_rate)(x)
        outputs = Dense(N_OUTPUTS, activation='linear', kernel_initializer='he_uniform')(x)
        model = keras.Model(inputs=input_x, outputs=outputs)
        return model

    return senet_nn

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 24, 39)            12324     
Total params: 12,324
Trainable params: 0
Non-trainable params: 12,324
_________________________________________________________________


In [33]:
emb_model = init_senet_w_emb()(24, 39)
emb_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 24, 39)]     0                                            
__________________________________________________________________________________________________
conv1d_8 (Conv1D)               (None, 24, 128)      55040       input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, 24, 128)      512         conv1d_8[0][0]                   
__________________________________________________________________________________________________
activation_8 (Activation)       (None, 24, 128)      0           batch_normalization_8[0][0]      
____________________________________________________________________________________________

### Compile Model 

In [34]:
emb_model.compile(optimizer='adam',loss='mse',metrics=[rmse, R2])

### Train Model 

In [35]:
PATIENCE = 10
BATCH_SIZE = 128
EPOCHS = 100

early = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=PATIENCE, verbose=1, mode='min', baseline=None, restore_best_weights=False)
callback_list = [early]

In [36]:
history = emb_model.fit(X_train,y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=2,
    validation_data=(X_val,y_val),
    callbacks=callback_list)

Train on 76260 samples, validate on 9340 samples
Epoch 1/100
76260/76260 - 25s - loss: 0.1123 - rmse: 0.3124 - R2: 0.5447 - val_loss: 0.0468 - val_rmse: 0.1984 - val_R2: 0.8105
Epoch 2/100
76260/76260 - 16s - loss: 0.0608 - rmse: 0.2280 - R2: 0.7529 - val_loss: 0.0367 - val_rmse: 0.1747 - val_R2: 0.8509
Epoch 3/100
76260/76260 - 16s - loss: 0.0525 - rmse: 0.2099 - R2: 0.7875 - val_loss: 0.0358 - val_rmse: 0.1717 - val_R2: 0.8546
Epoch 4/100
76260/76260 - 16s - loss: 0.0482 - rmse: 0.2006 - R2: 0.8047 - val_loss: 0.0341 - val_rmse: 0.1677 - val_R2: 0.8609
Epoch 5/100
76260/76260 - 16s - loss: 0.0455 - rmse: 0.1949 - R2: 0.8156 - val_loss: 0.0312 - val_rmse: 0.1616 - val_R2: 0.8731
Epoch 6/100
76260/76260 - 16s - loss: 0.0437 - rmse: 0.1907 - R2: 0.8230 - val_loss: 0.0317 - val_rmse: 0.1616 - val_R2: 0.8714
Epoch 7/100
76260/76260 - 17s - loss: 0.0422 - rmse: 0.1874 - R2: 0.8292 - val_loss: 0.0305 - val_rmse: 0.1597 - val_R2: 0.8760
Epoch 8/100
76260/76260 - 16s - loss: 0.0407 - rmse: 0.

In [37]:
emb_model.save('model_with_embedded.hdf5')

## 4.) Evaluation 

In [ ]:
emb_model = tf.keras.models.load_model('model_with_embedded.hdf5', custom_objects={'rmse': rmse, 'R2':R2})

In [ ]:
emb_model.evaluate(X_test,y_test,verbose=0)

In [ ]:
y_pred = emb_model.predict(X_test)

### Check error by each data 

In [ ]:
errs = compute_rmse(y_test,y_pred, axis=1)

In [ ]:
print(err)

### Error rank by highest 

In [ ]:
err_df = pd.DataFrame({'phonetic':z_test, 'err':errs})

In [ ]:
err_df.head()

In [ ]:
err_df.sort_values('err')

In [ ]:
err_df.hist('err')

In [ ]:
err_df_high = err_df[err_df['err'] > 0.15]

In [ ]:
err_df_high.head()

In [ ]:
err_df[err_df['err'] > 0.5].groupby('phonetic')['phonetic'].count().sort_values(ascending=False).plot.bar()

In [ ]:
err_df_high.groupby('phonetic')['phonetic'].count()

## Evaluate each param of each row

In [ ]:
se = compute_se(y_test,y_pred)
errs = compute_rmse(y_test,y_pred, axis=1)

In [ ]:
df_se = pd.DataFrame(se, columns=["HX","HY","JA","LP","LD","VS","VO","TCX","TCY","TTX","TTY","TBX","TBY","TS1","TS2","TS3","TS4"])

In [ ]:
df_se['phonetic'] = z_test
df_se['mean_err'] = errs
df_se['audio'] = a_test

In [ ]:
df_se.head()

In [ ]:
df_se[(df_se['phonetic'] == '7') & (df_se['mean_err'] > 0.5)]['TS4'].plot.hist()

## EDA audio of testing data 

In [ ]:
df_se_sort = df_se.sort_values('mean_err',ascending=False)
df_se_sort.reset_index(drop=True, inplace=True)

In [ ]:
df_se_sort[df_se_sort['phonetic'] == '&'].head(n=5)

In [ ]:
Audio(df_se_sort['audio'][0], rate=16000)

In [ ]:
df_se_sort[df_se_sort['phonetic'] == '&'].tail(n=5)

In [ ]:
Audio(df_se_sort['audio'][12073], rate=16000)